In [ ]:
# -*- coding: utf-8 -*-
import sys
import time
import json
from typing import Dict, List, Tuple, Set
import requests
import pandas as pd
import numpy as np

# Importar la librería deep_translate
from deep_translator import GoogleTranslator

import plotly.express as px
import streamlit as st

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperamos variables inyectadas por app.py
palette = locals().get("active_palette", ["#0576F3", "#36F48C", "#F47806"])
active_font = locals().get("active_font", "sans-serif")

# Parámetros principales del World Bank
WB_BASE = "https://api.worldbank.org/v2"
INDICATOR = "ST.INT.RCPT.CD"
TIMEOUT = 30
MAX_PER_PAGE = 20000
TOP_N = 15

# Mapeo de colores a los 3 años (Del más antiguo al más reciente)
def get_color_map_template(palette):
    p = palette + palette # Asegurar ciclo si la paleta es corta
    return {
        "OLDEST": p[2],  # Color 3 de la paleta original (Antiguo)
        "PREV": p[1],    # Color 2 de la paleta original (Medio)
        "RECENT": p[0]   # Color 1 de la paleta original (Reciente)
    }

COLOR_MAP_TEMPLATE = get_color_map_template(palette)

# -----------------------------------------------------------------------------
# 2. UTILIDADES DE DATOS Y TRADUCCIÓN (CON CACHÉ)
# -----------------------------------------------------------------------------

def get_json(url: str, params: Dict = None, max_retries: int = 3, backoff: float = 1.6):
    """Utilidad HTTP robusta."""
    sess = requests.Session()
    sess.headers.update({"User-Agent": "StreamlitApp/WorldBank-WDI (python requests)"})
    params = params or {}
    for attempt in range(1, max_retries + 1):
        try:
            resp = sess.get(url, params=params, timeout=TIMEOUT)
            resp.raise_for_status()
            return resp.json()
        except Exception:
            if attempt == max_retries: raise
            time.sleep(0.5)

@st.cache_data(show_spinner=False)
def fetch_paged(url: str, params: Dict) -> List[Dict]:
    """Descarga paginada de la API del Banco Mundial."""
    params = dict(params)
    params.setdefault("format", "json")
    params.setdefault("per_page", MAX_PER_PAGE)
    data = get_json(url, params=params)
    if not isinstance(data, list) or len(data) < 2: return []

    results = data[1]
    pages = int(data[0].get("pages", 1))
    for page in range(2, pages + 1):
        params["page"] = page
        data = get_json(url, params=params)
        results.extend(data[1])
    return results

@st.cache_data(show_spinner=False)
def fetch_countries_non_aggregates() -> Tuple[Set[str], Dict[str, str]]:
    """Obtiene el catálogo de países sin agregados regionales."""
    url = f"{WB_BASE}/country"
    params = {"format": "json", "per_page": 400}
    data = get_json(url, params=params)
    if not isinstance(data, list) or len(data) < 2: return set(), {}

    iso3_set = set()
    iso3_to_name = {}
    for c in data[1]:
        region = c.get("region", {}) or {}
        if region.get("id") == "NA" or "Aggregate" in str(region.get("value", "")): continue
        iso3 = c.get("id")
        name = c.get("name")
        if iso3 and name:
            iso3_set.add(iso3)
            iso3_to_name[iso3] = name
    return iso3_set, iso3_to_name

@st.cache_data(show_spinner=False)
def fetch_indicator_all() -> pd.DataFrame:
    """Descarga todos los datos del indicador ST.INT.RCPT.CD."""
    url = f"{WB_BASE}/country/all/indicator/{INDICATOR}"
    rows = fetch_paged(url, params={"format": "json", "per_page": MAX_PER_PAGE})
    recs = []
    for r in rows:
        recs.append({
            "iso3": r.get("countryiso3code"),
            "country": (r.get("country") or {}).get("value"),
            "year": int(r.get("date")),
            "value": float(r.get("value")) if r.get("value") is not None else None
        })
    return pd.DataFrame.from_records(recs)

# =============================================================================
# >>> FUNCIÓN DE TRADUCCIÓN (CACHÉ ACTIVO) <<<
# =============================================================================
@st.cache_data(show_spinner="Traduciendo nombres automáticamente...")
def translate_countries_deep_translate(country_names: List[str]) -> Dict[str, str]:
    """Traduce una lista de nombres de países al español usando deep_translate."""
    
    # 1. Inicializar traductor
    try:
        translator = GoogleTranslator(source='en', target='es')
    except Exception as e:
        raise Exception(f"Fallo inicialización traductor: {e}")

    # 2. Traducir lote
    try:
        translated_list = translator.translate_batch(country_names)
    except Exception as e:
        raise Exception(f"Error en servicio de traducción: {e}")

    # 3. Crear mapa
    translation_map = {}
    for original, translated in zip(country_names, translated_list):
        if translated and original.lower() != translated.lower():
            translation_map[original] = translated
        else:
            translation_map[original] = original
            
    return translation_map

# -----------------------------------------------------------------------------
# 3. LÓGICA PRINCIPAL Y VISUALIZACIÓN
# -----------------------------------------------------------------------------

def plot_ranking_chart(melt_df, country_order, years_for_plot, is_translated: bool):
    """Genera y muestra la gráfica Plotly."""

    # 1. Mapeo de colores dinámico
    color_discrete_map = {
        str(years_for_plot[0]): COLOR_MAP_TEMPLATE["OLDEST"],
        str(years_for_plot[1]): COLOR_MAP_TEMPLATE["PREV"],
        str(years_for_plot[2]): COLOR_MAP_TEMPLATE["RECENT"]
    }

    # 2. Crear la figura
    fig = px.bar(
        melt_df,
        x="Ingresos",
        y="country",
        color="Año",
        orientation="h",
        color_discrete_map=color_discrete_map,
        barmode='stack'
    )

    # 3. Personalización de Plotly
    # Simplificamos el título asumiendo que casi siempre estará traducido
    title_suffix = "" if is_translated else " (Nombres en inglés)"
    
    fig.update_layout(
        title=dict(
            text=f"Top {TOP_N} países por ingresos de divisas de turismo internacional ({years_for_plot[0]}-{years_for_plot[-1]}){title_suffix}",
            x=0.5, xanchor='center',
            font=dict(size=18)
        ),
        xaxis_title="Gasto de turistas internacionales (USD)",
        yaxis_title="",
        font=dict(family=active_font, size=12),
        legend_title_text='Año',
        hoverlabel=dict(bgcolor="white", font_size=12, font_family=active_font),
        height=700,
        template="plotly_white",
        margin=dict(t=80, b=100)
    )

    # 4. Formato del eje X (Valores en US$ corrientes)
    fig.update_xaxes(
        tickformat="$,.0f",
        showgrid=True,
        gridcolor='#e0e0e0',
    )

    # Tooltip
    fig.update_traces(
        hovertemplate="<b>%{y}</b><br>Año %{customdata[0]}: $%{x:,.0f} USD<extra></extra>",
        customdata=np.stack([melt_df['Año']], axis=-1)
    )

    # Invertir el eje Y para que el top 1 esté arriba
    fig.update_yaxes(categoryorder="array", categoryarray=country_order[::-1])

    # Anotaciones de fuente
    source_txt = "Fuente: World Bank (WDI) – indicador ST.INT.RCPT.CD."
    notes_txt = f"Notas: Top {TOP_N} definido por el último año con datos; montos en US$ corrientes."

    fig.add_annotation(text=source_txt, xref="paper", yref="paper", x=0, y=-0.15, showarrow=False, font=dict(size=10, color="gray"), align="left")
    fig.add_annotation(text=notes_txt, xref="paper", yref="paper", x=0, y=-0.2, showarrow=False, font=dict(size=10, color="gray"), align="left")

    st.plotly_chart(fig, use_container_width=True)


def main_flow():
    # 1. Descarga y procesamiento de datos
    try:
        valid_iso3, iso3_to_name = fetch_countries_non_aggregates()
        df = fetch_indicator_all()
    except Exception as e:
        st.error(f"❌ Error al descargar datos del Banco Mundial: {e}")
        return

    # 2. Filtrado y determinación de años
    df = df.dropna(subset=["value"]).copy()
    df = df[df["iso3"].isin(valid_iso3)].copy()

    years_sorted_desc = sorted(df["year"].unique(), reverse=True)
    last3_years = years_sorted_desc[:3]
    if len(last3_years) < 3: return

    y_latest = last3_years[0]
    years_for_plot = sorted(last3_years)

    # Determinar el Top N y consolidar
    top_iso3 = df[df["year"] == y_latest].sort_values("value", ascending=False).head(TOP_N)["iso3"].tolist()

    rows = []
    for iso in top_iso3:
        row = {"iso3": iso, "country": iso3_to_name.get(iso, iso)}
        for y in years_for_plot:
            val = df.loc[(df["iso3"] == iso) & (df["year"] == y), "value"]
            row[str(y)] = float(val.iloc[0]) if not val.empty else 0.0
        rows.append(row)
    
    # DataFrame Base
    mat_df = pd.DataFrame(rows).sort_values(str(y_latest), ascending=False)

    # --- LÓGICA DE TRADUCCIÓN AUTOMÁTICA ---
    country_list_en = mat_df["country"].tolist()
    is_translated = False
    
    # Intentamos traducir automáticamente
    try:
        # La función está cacheada, así que esto es rápido en re-runs
        translation_map = translate_countries_deep_translate(country_list_en)
        if translation_map:
            mat_df["country"] = mat_df["country"].apply(lambda x: translation_map.get(x, x))
            is_translated = True
    except Exception as e:
        # Si falla (ej. sin internet), mostramos warning pero seguimos con nombres en inglés
        st.warning(f"⚠️ No se pudo traducir automáticamente (mostrando en inglés). Error: {e}")
        is_translated = False

    # 3. Preparar DataFrame Largo (Melt) para Plotly
    melt_df = mat_df.melt(
        id_vars=["iso3", "country"],
        value_vars=[str(y) for y in years_for_plot],
        var_name="Año",
        value_name="Ingresos"
    )

    # Orden final para la gráfica
    country_order = mat_df["country"].tolist()

    # 4. Generar la Gráfica
    plot_ranking_chart(
        melt_df,
        country_order,
        years_for_plot,
        is_translated
    )

    # =============================================================================
    # 5. TABLA DE DATOS DETALLADA (MODIFICACIÓN SOLICITADA)
    # =============================================================================
    st.markdown("### 📊 Datos Detallados (Top Países)")

    # Preparamos el DataFrame para la tabla
    # Seleccionamos las columnas: País y los años
    cols_years = [str(y) for y in years_for_plot]
    table_df = mat_df[["country"] + cols_years].copy()
    
    # Renombramos la columna country a País (ya está traducida en mat_df)
    table_df = table_df.rename(columns={"country": "País"})

    # Configuración de columnas para formatear moneda
    column_config = {
        "País": st.column_config.TextColumn("País"),
    }
    for col in cols_years:
        column_config[col] = st.column_config.NumberColumn(
            label=col,
            format="$%,.0f" # Formato moneda USD sin decimales
        )

    st.dataframe(
        table_df,
        use_container_width=True,
        hide_index=True,
        column_config=column_config
    )

# -----------------------------------------------------------------------------
# 4. PUNTO DE ENTRADA
# -----------------------------------------------------------------------------

if __name__ == "__main__":
    st.markdown("### 🌎 Ranking Mundial de Divisas Turísticas")
    main_flow()